# 2.1 Spark Concepts

[Spark](http://spark.apache.org/) is a framework for distributed computation.  Originally written in `Scala`, it also has complete bindings in Python and Java.  Bindings in `R` are available but are a work-in-progress.

Since distributed computing is **very** different from computation in a single machine with modest data sizes, let's go over some conceptual differences first before digging into practical work.

### 2.1.1 Distributed Computing Concepts

"Distributed computing" roughly means "coordinating multiple computers (a **cluster**) to solve a large problem".  Each computer has dedicated memory, CPUs and disks, as well as network connections to all other computers.

Why bother at all?  Why not just **scale vertically**, i.e., buy a bigger machine for SQL Server?
1. You quickly hit **economic** barriers to scaling.  Having 1 PB of RAM on a single machine is much more expensive than 100GB of RAM on 10 machines.  Same for hard disks, multicore CPUs, network bandwidth, etc.
2. Scaling is **inelastic**.  If you want to double the RAM on your SQL Server machine, you typically have to (a) buy higher-density RAM to replace *all* the RAM because your machine has limited capacity, and (b) turn off the machine while changing the RAM chips.  Scaling up and down with daily / weekly / monthly usage is almost unthinkable.
3. **Component failure can be catastrophic**.  If the air conditioning on your 128-core CPU fails, the CPU can burst into flames and your entire database becomes inaccessible.

Instead, at large scales, you typically **scale horizontally**, meaning you build a cluster of machines.  The costs per unit data and compute speed are then roughly constant, and component failure can be worked around.

Conceptually, this is how a cluster is organized:

<img src="images/Hadoop-Cluster-Architecture-and-Components-1.jpg" width="70%">  
_Image source: [SAP Hana Tutorial](http://saphanatutorial.com/hadoop-cluster-architecture-and-core-components/)_

Physically, this is what a large cluster looks like:

<img src="images/Hadoop-Cluster-Architecture-and-Components-3.jpg" width="70%">  
_Image source: [SAP Hana Tutorial](http://saphanatutorial.com/hadoop-cluster-architecture-and-core-components/)_

The cluster's architecture has lots of consequences for how distributed computations must be set up.  Here are the highlights:

1. **Any one machine only holds a small portion of the data.**  Typically, this might be a few 100 GBs to a few TBs on disk, much less in memory.  So **you can't hold all the data in memory at once**, at least not on a single machine.  Global operations that reorganize data involve actually moving data over the network.  This is not too fast within a rack, and really slow between racks.<br><br>

2. Compared to the data, program code is typically tiny.  So it's more efficient to **send the computation to the data* than *to download the data to a single computer**.  Hence, your code will execute in different environments simultaneously: **there are no shared variables or global state.**<br><br>

3. In a large cluster of machines, **failure is ubiquitous**, so calculations need to detect and recover from component failure.  Things that seem practically impossible to you on a laptop are actually common in a cluster.  For instance:
  - Individual hard disks in a cluster may die at a rate of 1-10s disks / day.
  - Heat and cosmic rays will corrupt a few bytes of your data here and there every day (i.e., data actually rots)
  - Entire sections of your cluster will fail simultaneously (e.g., network switches die, power units die, technicians make errors, ...)
  - Memory, CPUs, network cards, etc. will all fail routinely.

#### So what does this mean for me?

The first two points really change how you write computations and which algorithms are efficient.  For example:
- To avoid network traffic, it really pays to filter out any data you don't need for your calculation as early as possible.
- To simplify sending calculations to the data instead of vice-versa, you end up writing code in a style similar to _functional programming_.

Here's an example of typical code that works fine on a small scale but is hard to distribute transparently:

In [1]:
# Calculate the sum of all odd squares in the set {1^2, ..., 10^2}

squares = []             # This variable is shared across loop iterations
for i in range(1,10+1):
    squares.append(i**2) # Each loop iteration must run in order for the
                         # the result to be correct

odd_squares = []         # squares is an intermediate value, but we've
for x in squares:        # materialized it as a whole list before
    if x % 2 == 1:       # filtering on only the odd squares
        odd_squares.append(x)
        
total = 0
for x in odd_squares:
    total = total + x

total

165

Instead, if we use list comprehensions and built-in reduction operations, we give the underlying implementation a chance to distribute the calculation automatically:

In [2]:
odd_squares = [i**2 for i in range(1,10+1) if i**2 % 2 == 1]
# Python could choose to calculate each item in squares
# in a different machine and assemble the results.
# It also doesn't have to store all the squares from 1^2 to 10^2
# before selecting only the odd ones.

total = sum(odd_squares)
# Internally, sum could add subsets of squares in different machines
# and assemble the final result later

total

165

Spark forces you to write your calculations much more like the second example.  In return, your calculation gets split up and run on a cluster without you having to worry about the details.

As for failure (point 3 above), `Spark` detects and recovers from failure transparently, so you don't need to worry about that (phew!).  But Spark's mechanism for handling failure dictates the structure and behaviour of its central abstraction (**Resilient Distributed Datasets** or RDDs)

#### Enough theory, let's get our hands dirty!!

`Spark` should be installed on your laptops.  Whether you're running on a single-CPU laptop or on a 4000-node cluster, `Spark` code looks the same.  So today, we'll run things in your laptop.  Tomorrow, we'll use a cluster built in the AWS cloud.

**Ex 2.1.1 Running `pyspark` in a shell**
* **Open `cmd.exe` and go to `C:\Users\<youruser>\training` (`cd training`).**
* **Type `dir` to find the `pyspark` installation** (it should be something like `spark-1.2.0-bin-hadoop2.4`).
* **Go into the spark directory (`cd spark-1.2.0-bin-hadoop2.4`)**
* **Type `bin\pyspark`**

You should see lots of messages scroll by, and eventually, some text like this:
```
Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 1.2.0
      /_/

Using Python version 2.7.9 (v2.7.9:648dcafa7e5f, Dec 10 2014 10:10:46)
SparkContext available as sc.
>>> 
```

We'll exercise `Spark` with a simple example to make sure that everything is working.  Don't worry if the following doesn't yet make sense to you:

**Ex 2.1.2.  Type the following commands into `pyspark`:**
```
rdd = sc.parallelize([1,2,3,4,5])
rdd.count()
```

You should again see lots of messages again, but you should see the answer `5` just above the `>>>` prompt when it finally appears.

Press Ctrl-D or type `quit()` to exit the `pyspark` shell.

**What's happening?**  `pyspark` launched a `python` shell.  Before giving you a prompt, `pyspark` ran some code to start a local `Spark` server over all the CPUs on your machine, then connected to it.  It made the connection available in Python via the variable `sc`.

Just like normal `python`, we can do much better than the standard `python` shell.  We can run `pyspark` inside an `ipython` shell or even inside an `IPython notebook`.

**Ex 2.1.3 Running `pyspark` in an `ipython` shell.**
* **Open `cmd.exe` and navigate to the Spark installation as before.**
* **Type `set IPYTHON=1`**
* **Type `bin\pyspark`**  

**Now type the same commands on `rdd` as before.  Verify that you get the answer `5`**

**Ex 2.1.4 Running `pyspark` in an IPython notebook.**
* **Open `cmd.exe` and navigate to the Spark installation as before.**
* **Type `set IPYTHON=1`**
* **Type `set IPYTHON_OPTS=notebook`**
* **Type `bin\pyspark`**

**Now create a new notebook.  In it, make a cell with the same commands on `rdd` as before and evaluate it.  Verify that you get the answer `5`**

Note that you don't see the large stream of messages in the IPython Notebook.  If you look closely, you'll see the same messages have appeared in the window with `cmd.exe` in it.

Also, if you try to type the `rdd` commands in your existing IPython Notebook, they won't work.  That's because you opened that IPython notebook directly, without giving pyspark to run its initialization code.

**Risky:** If you want, you can run the following snippet in any open IPython notebook to launch Spark locally and connect to it.  I've tested it on my machine, and I think it should work on your machine, but haven't tested it there!

In [3]:
# These depend on your environment
import os

if os.sys.platform == 'win32':
    SPARK_HOME = os.path.join(os.environ['USERPROFILE'],
                              'training',
                              'spark-1.2.0-bin-hadoop2.4'
                             )
else:
    SPARK_HOME = '/Users/pat/Programs/spark-1.2.0-bin-hadoop2.4'

JAVA_HOME = os.environ.get(
    'JAVA_HOME',
    '/Library/Java/JavaVirtualMachines/jdk1.8.0_31.jdk/Contents/Home'
)

# Setting up a local Spark context in IPython is
# a bit harder than it should be...
import os
import sys

os.environ['SPARK_HOME'] = SPARK_HOME
sys.path.append(JAVA_HOME)
subpaths = [
    'bin',
    'python',
    os.path.join('python', 'pyspark'),
    os.path.join('python', 'lib'),
    os.path.join('python', 'lib', 'pyspark.zip'),
    os.path.join('python', 'lib', 'py4j-0.8.2.1-src.zip'),
    'python'
]
for subpath in subpaths:
    sys.path.append(os.path.join(os.environ['SPARK_HOME'], subpath))
    
from pyspark import SparkContext
from pyspark import SparkConf

sc = SparkContext(master='local[*]', appName='test')

After running that snippet, you can write Spark code as before:

In [4]:
rdd = sc.parallelize([1,2,3,4,5])
rdd.count()

5

Finally, as with Python, you can write large PySpark scripts in a text file that you then submit to a cluster.  The only caveat is that you must initialize the `SparkContext` yourself.

**Ex 2.1.5 Running a PySpark script**
* **Create a new file `pysparkscript.py` with the following contents:**

```
from pyspark import SparkContext

sc = SparkContext(master='local[*]', appName='MyPySparkScript')
rdd = sc.parallelize([1,2,3,4,5])
print(rdd.count())
```

* **Run the script by typing the following command in `cmd.exe`:**
```
C:\PATH\TO\SPARK\FOLDER\bin\spark-submit pysparkscript.py
```

---

**NOTE:** For the remainder of the course, you should view the notebooks with an IPython notebook started by `pyspark`.  See Ex 2.1.4 for details.

---
# 15-20 minute break